<a href="https://colab.research.google.com/github/ramapriyakp/Portfolio/blob/master/NLP/Discovering_and_Visualizing_Topics_in_Texts_with_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Discovering and Visualizing Topics in Texts
The most classic applications of Natural Language Processing involve supervised Machine Learning. In the most typical cases of text classification, named entity recognition, question answering, etc., NLPers have access to a collection of texts with their labels. However, in real-life scenario's, we're often less lucky. Many text collections do not come with metadata labels that tell you what the texts are about. When people answer open-ended survey questions, for example, they don't tag their answer with the topics they discuss. In such cases, we can make use of unsupervised techniques we call topic models.

Topic models are a family of models that are able to discover the topics in a collection of texts. In this context, "topics" refers to groups of related words that often occur together in the same text. For example, in a collection of newspaper articles a topic model may identify one topic that is made up of words such as "politician", "law", and "parliament", and another characterized by words such as "player", "match" and "penalty". Topic models can only find such clusters of related words; it is our task as humans to interpret these topics and give them labels such as "politics" and "football".

One of the most popular such models is Latent Dirichlet Allocation (LDA). LDA is a generative model that sees every text as a mixture of topics. Each of these topics are responsible for some of the words in the text. For example, the "football" topic will generate the word "penalty" with a high probability, while the "politics" topic will have a much higher probability for "politician" than for "penalty". Other words, such as "the" and "an", will have similar probabilities in all topics. LDA takes its name from the Dirichlet probability distribution. This is the prior distribution it assumes the topics in a text will have.

## Data
One of the contexts where topic modelling is extremely useful is that of open-ended survey questions. It allows us to explore the variation in topics that people's answers contain. As our example data set, let's therefore take a look at an extensive set of answers from the Grand Débat National in France, the public debate organized by president Macron. The goal of the debate was to better understand the French people's needs and opinions after the mass demonstrations of the Yellow vests movement. The results of this debate are now available as open data. For our experiments, we'll download one of the csv files about the ecological transition and load the contents into a pandas dataframe

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/NLP

/content/drive/My Drive/NLP


In [0]:
!wget http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net/2019-01-31/LA_TRANSITION_ECOLOGIQUE.csv

--2019-09-05 04:28:21--  http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net/2019-01-31/LA_TRANSITION_ECOLOGIQUE.csv
Resolving opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net (opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net)... 164.132.114.175
Connecting to opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net (opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net)|164.132.114.175|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64239619 (61M) [text/csv]
Saving to: ‘LA_TRANSITION_ECOLOGIQUE.csv’

LA_TRANSITION_ECOLO 100%[===================>]  61.26M  45.7MB/s    in 1.3s    

2019-09-05 04:28:23 (45.7 MB/s) - ‘LA_TRANSITION_ECOLOGIQUE.csv’ saved [64239619/64239619]



In [0]:
%pylab inline
import pandas as pd

f = "LA_TRANSITION_ECOLOGIQUE.csv"
df = pd.read_csv(f)

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Each of the rows in this data frame contains some metadata and a respondent's answers to a list of questions about the ecological transition. 
Some of these questions are multiple choice, while other ones are open-ended.

In [0]:
df.columns

Index(['reference', 'title', 'createdAt', 'publishedAt', 'updatedAt',
       'trashed', 'trashedStatus', 'authorId', 'authorType', 'authorZipCode',
       'Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?',
       'Que faudrait-il faire selon vous pour apporter des réponses à ce problème ?',
       'Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?',
       'Si oui, de quelle manière votre vie quotidienne est-elle touchée par le changement climatique ?',
       'À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?',
       'Si oui, que faites-vous aujourd'hui pour protéger l'environnement et/ou que pourriez-vous faire ?',
       'Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou renoncer à prendre votre véhicule pour de très petites distances ?',
   

We'll focus on the last of the questions, which gives the most freedom to the respondents: it asks them whether they have any additional comments about the ecological transition. We hope LDA will help us analyze what topics their answers focus on. The first few answers to this question already give us an idea of the variety of topics people bring up: alternative energy sources ("les centrales géothermiques"), politics ("une vrai politique écologique") and education ("pédagogie").

In [0]:
question = "Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?"
df[question].head(10)

0               Multiplier les centrales géothermiques
1    Les problèmes auxquels se trouve confronté l’e...
2                                                  NaN
3                                                  NaN
4      Une vrai politique écologique et non économique
5    Les bonnes idées ne grandissent que par le par...
6    Pédagogie dans ce sens là dés la petite école ...
7                                                  NaN
8    faire de l'écologie incitative et non punitive...
9    Développer le ferroutage pour les poids lourds...
Name: Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?, dtype: object

## Preprocessing
Before we train a topic model, we need to tokenize our texts. Let's do this with the spaCy NLP library. Because we're only going to do some basic preprocessing, we don't need to download any of its statistical models. We'll just initialize a blank model for French instead.

In [0]:
import spacy
nlp = spacy.blank("fr")

First we remove all the rows from the data frame that don't have a response for our target question (the NaNs above), then we take all the texts in the target column. Next, we use spaCy to perform our first preprocessing pass.

In [0]:
texts = df[df[question].notnull()][question]
%time spacy_docs = list(nlp.pipe(texts))

CPU times: user 21.4 s, sys: 320 ms, total: 21.8 s
Wall time: 21.8 s


Now that we have a list of spaCy documents, we transform them to lists of tokens. Instead of the original tokens, we're going to work with the lemmas instead. This will allow our model to generalize better, as it will be able to see that "géothermiques" and "géothermique" are actually just two forms of the same words. This is the full list of our initial preprocessing steps:


*   we remove all words shorter than 3 characters (these are often fairly uninteresting from a topical point of view),
*   we drop all stopwords, and
*   we take them lemmas of the remaining words and lowercase them.



In [0]:
docs = [[t.lemma_.lower() for t in doc if len(t.orth_) > 3 and not t.is_stop] for doc in spacy_docs]
print(docs[:3])

[['multiplier', 'central', 'géothermique'], ['problème', 'trouver', 'confronter', 'ensemble', 'planète', 'dénoncer', 'parfait', 'désordre', 'gilet', 'jaune', 'france', 'surpopulation', 'mondial', 'population', 'passée', 'd’1,5', 'milliard', 'habitant', '1900', 'milliard', '2020', 'monter', 'bientôt', 'milliard', '2040', 'progrès', 'communication', 'village', 'mondial', 'individu', 'fond', 'asie', 'fond', 'afrique', 'passant', 'quartier', 'campagne', 'pays', 'aspirer', 'vivre', 'blâmer', 'lotir', 'concitoyen', 'logement', 'nourriture', 'bien', 'consommation', 'déplacement', 'mère', 'problème', 'solution', 'problème', 'stabilisation', 'croissance', 'démographique', 'partage', 'richesse', 'partage', 'terrer', 'partage', 'protection', 'biodiversité', 'règlement', 'conflit', 'lutte', 'déforestation', 'lutte', 'dérèglement', 'climatique', 'règlement', 'conflit', 'stabilisation', 'migration', 'concurrence', 'commerciale', 'mondial', 'etc.', 'français', 'européen', 'mondial', 'france', 'jouer'

Next, we also want to take frequent bigrams into account. After all, French has many multiword units, such as "poids lourds" (trucks) that actually form one word rather than two. This is the first step where we use the Gensim library, a great NLP library for topic modelling. First we identify the frequent bigrams in the corpus, then we append them to the list of tokens for the documents in which they appear. This means the bigrams will not be in their correct position in the text, but that's fine: topic models are bag-of-word models that ignore word position anyway.

In [0]:
import re
from gensim.models import Phrases

bigram = Phrases(docs, min_count=10)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:  # bigrams can be recognized by the "_" that joins the invidual words
            docs[idx].append(token)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
docs[2]

['vrai', 'politique', 'écologique', 'économique', 'vrai_politique']

Next, we move on to the final Gensim-specific preprocessing steps. First, we create a dictionary representation of the documents. This dictionary will map each word to a unique ID and help us create bag-of-word representations of each document. These bag-of-word representations contain the ids of the words in the document, together with their frequency. Additionally, we can remove the least and most frequent words from the vocabulary. This improves the quality of our topic model and speeds up its training. The minimum frequency of a word is expressed as an absolute number, the maximum frequency is the proportion of documents a word is allowed to occur in.

In [0]:
from gensim.corpora import Dictionary

dictionary = Dictionary(docs)
print('Number of unique words in original documents:', len(dictionary))

dictionary.filter_extremes(no_below=3, no_above=0.25)
print('Number of unique words after removing rare and common words:', len(dictionary))

print("Example representation of document 3:", dictionary.doc2bow(docs[2]))

Number of unique words in original documents: 31035
Number of unique words after removing rare and common words: 12076
Example representation of document 3: [(85, 1), (86, 1), (87, 1), (88, 1), (89, 1)]


In [0]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

##Training
Now it's time to train our topic model. We do this with the following parameters:


*   corpus: the bag-of-word representations of our documents
*   id2token: the mapping from indices to words
*   num_topics: the number of topics we want the model to identify
*   chunksize: the number of documents the model sees for every update
*   passes: the number of times we show the total corpus to the model during training
*   random_state: we use a seed to ensure reproducibility.


On a corpus of this size, the training will typically take one or two minutes.

In [0]:
from gensim.models import LdaModel

%time model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, chunksize=1000, passes=5, random_state=1)

CPU times: user 44 s, sys: 994 ms, total: 45 s
Wall time: 43.9 s


## Results
Let's take a look at what the model has learnt. We do this by printing out the ten words that are most characteristic for each of the topics. This shows some interesting patterns already: while some topics are more general (such as 3), others point to some very relevant recurring themes: electric vehicles (topic 1), (alternative) energy (topic 2), agriculture (topic 6), waste and recycling (topic 7) and taxes (topic 9).

In [0]:
for (topic, words) in model.print_topics():
    print(topic+1, ":", words)

1 : 0.030*"emballage" + 0.027*"plastique" + 0.023*"déchet" + 0.010*"espace" + 0.009*"consigne" + 0.009*"industriel" + 0.009*"recyclable" + 0.009*"falloir" + 0.008*"interdire" + 0.008*"gaspillage"
2 : 0.061*"produit" + 0.022*"agriculture" + 0.016*"interdire" + 0.013*"pesticide" + 0.012*"production" + 0.012*"agricole" + 0.012*"agriculteur" + 0.012*"favoriser" + 0.011*"santé" + 0.010*"local"
3 : 0.040*"transport" + 0.019*"taxer" + 0.018*"ville" + 0.014*"router" + 0.013*"camion" + 0.010*"vitesse" + 0.009*"voiture" + 0.009*"développer" + 0.009*"grand" + 0.008*"centre"
4 : 0.012*"politique" + 0.011*"pays" + 0.011*"terme" + 0.011*"france" + 0.010*"climatique" + 0.009*"problème" + 0.009*"mondial" + 0.008*"question" + 0.007*"grand" + 0.006*"long"
5 : 0.039*"voiture" + 0.037*"électrique" + 0.035*"véhicule" + 0.014*"voiture_électrique" + 0.014*"diesel" + 0.012*"batterie" + 0.011*"solution" + 0.010*"automobile" + 0.009*"polluant" + 0.009*"moteur"
6 : 0.016*"public" + 0.014*"pouvoir" + 0.013*"trava

In [0]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 6.3MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=2e8a828ba1ccff4309a766ce6874027109babc776ff073f6f613fda91954dda3
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


Another way of inspecting the topics is by visualizing them. This can be done with the pyLDAvis library. PyLDAvis will show us how popular the topics are in our corpus, how similar the topics are, and which are the most salient words for this topic. Note it's important to set sort_topics=False on the call to pyLDAvis. If you don't, it will order the topics differently than Gensim.

In [0]:
import pyLDAvis.gensim
import warnings

pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.181738  0.133551       1        1   5.531366
1     -0.176970  0.019171       2        1   9.642430
2     -0.122157 -0.067595       3        1   9.807094
3      0.241860  0.127069       4        1  11.707786
4      0.070409 -0.058880       5        1  10.792009
5      0.040708 -0.238380       6        1   8.697709
6      0.113342 -0.025149       7        1  16.690987
7      0.210763  0.167094       8        1  10.278959
8      0.007686 -0.195857       9        1  10.556250
9     -0.203903  0.138976      10        1   6.295412, topic_info=     Category         Freq  ...  loglift  logprob
255   Default  5106.000000  ...  30.0000  30.0000
533   Default  4646.000000  ...  29.0000  29.0000
456   Default  4520.000000  ...  28.0000  28.0000
611   Default  3138.000000  ...  27.0000  27.0000
422   Default  3913.000000  ...  26.0000  26.0000
167   Default  3033.000000  ...  25.0000  25.0000
251   Default  3184.000000  ...  24.0000  24.0000
88    Default  5873.000000  ...  23.0000  23.0000
164   Default  2924.000000  ...  22.0000  22.0000
266   Default  1832.000000  ...  21.0000  21.0000
457   Default  2782.000000  ...  20.0000  20.0000
249   Default  2736.000000  ...  19.0000  19.0000
184   Default  6491.000000  ...  18.0000  18.0000
464   Default  1567.000000  ...  17.0000  17.0000
936   Default  1260.000000  ...  16.0000  16.0000
298   Default  1703.000000  ...  15.0000  15.0000
215   Default  1826.000000  ...  14.0000  14.0000
362   Default  1118.000000  ...  13.0000  13.0000
117   Default  7213.000000  ...  12.0000  12.0000
288   Default  2843.000000  ...  11.0000  11.0000
819   Default  1115.000000  ...  10.0000  10.0000
247   Default  1328.000000  ...   9.0000   9.0000
622   Default  1310.000000  ...   8.0000   8.0000
662   Default  2384.000000  ...   7.0000   7.0000
419   Default  1660.000000  ...   6.0000   6.0000
257   Default  1253.000000  ...   5.0000   5.0000
761   Default  2027.000000  ...   4.0000   4.0000
192   Default  2519.000000  ...   3.0000   3.0000
536   Default  2126.000000  ...   2.0000   2.0000
0     Default  1202.000000  ...   1.0000   1.0000
...       ...          ...  ...      ...      ...
1972  Topic10    99.932465  ...   2.7561  -6.1637
2975  Topic10    97.168472  ...   2.7559  -6.1918
3528  Topic10    96.421188  ...   2.7558  -6.1995
1474  Topic10    95.617043  ...   2.7557  -6.2079
4387  Topic10    95.302589  ...   2.7557  -6.2112
7099  Topic10    90.598709  ...   2.7552  -6.2618
266   Topic10  1777.613037  ...   2.7347  -3.2852
898   Topic10   616.512756  ...   2.6873  -4.3442
3238  Topic10   446.667114  ...   2.6004  -4.6664
1138  Topic10   695.822327  ...   2.4415  -4.2231
197   Topic10   512.085754  ...   2.4476  -4.5298
3773  Topic10   216.455917  ...   2.5849  -5.3909
249   Topic10  1398.381592  ...   2.0938  -3.5252
275   Topic10   646.449768  ...   2.1878  -4.2967
331   Topic10   615.321838  ...   2.0885  -4.3461
4693  Topic10   185.502426  ...   2.5071  -5.5452
1962  Topic10   230.478958  ...   2.2680  -5.3281
2336  Topic10   161.636459  ...   2.4689  -5.6829
14    Topic10   286.150574  ...   1.7133  -5.1117
144   Topic10   393.041290  ...   1.2199  -4.7943
761   Topic10   384.839142  ...   1.1035  -4.8154
37    Topic10   260.991669  ...   1.6359  -5.2038
288   Topic10   392.873749  ...   0.7859  -4.7948
794   Topic10   355.359009  ...   0.9113  -4.8951
227   Topic10   344.423309  ...   0.8819  -4.9264
71    Topic10   237.198807  ...   1.7238  -5.2993
2853  Topic10   237.225708  ...   1.7057  -5.2992
615   Topic10   226.158951  ...   1.8364  -5.3470
184   Topic10   345.988708  ...  -0.1665  -4.9218
228   Topic10   229.330872  ...   1.5659  -5.3331

[644 rows x 6 columns], token_table=      Topic      Freq                    Term
term                                         
3715      6  0.989756                  \n    
8530      6  0.994565             

Finally, let's inspect the topics the model recognizes in some of the individual documents. Here we see how LDA tends to assign a high probability to a low number of topics for each documents, which makes its results very interpretable.

In [0]:
for (text, doc) in zip(texts[:20], docs[:20]):
    print(text)
    print([(topic+1, prob) for (topic, prob) in model[dictionary.doc2bow(doc)] if prob > 0.1])

Multiplier les centrales géothermiques
[(5, 0.30585632), (9, 0.49412858)]
Les problèmes auxquels se trouve confronté l’ensemble de la planète et que dénoncent, dans le plus parfait désordre, les gilets jaunes de France ne sont-ils pas dus, avant tout, à la surpopulation mondiale ? Cette population est passée d’1,5 milliards d’habitants en 1900 à 7 milliards en 2020 et montera bientôt à 10 milliards vers 2040.  Avec les progrès de la communication dans ce village mondial, chaque individu, du fin fond de l’Asie au fin fond de l’Afrique, en passant par les « quartiers » et les « campagnes » de notre pays, aspire à vivre – et on ne peu l’en blâmer – comme les moins mal lotis de nos concitoyens (logement, nourriture, biens de consommation, déplacement,etc.).  Voilà la mère de tous les problèmes. Si tel est bien le cas, la solution à tous les problèmes (stabilisation de la croissance démographique, partage des richesses, partage des terres, partage de l’eau, protection de la biodiversité, rè

## Conclusions
Many collections of unstructured texts don't come with any labels. Topic models such as Latent Dirichlet Allocation are a useful technique to discover the most prominent topics in such documents. Gensim makes training these topics model easy, and pyLDAvis presents the results in a visually attractive way. Together they form a powerful toolkit to better understand what's inside large sets of documents, and to explore subsets of related texts. While these results are often very revealing already, it's also possible to use them as a starting point, for example for a labelling exercise for supervised text classification. Although traditional topic models are lacking in more semantic information (they don't use word embeddings, for instance), they should be in every NLPer's toolkit as a really quick way of getting insights into large collections of documents.